In [1]:
import re
import requests
import time
import pandas as pd
import logging
from datetime import datetime

In [159]:
url = 'https://ballotpedia.org/Supreme_Court_cases,_October_term_2024-2025'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)

assert response.status_code == 200, 'Статус респонса не 200'

html = response.content.decode()

blocks = re.findall(
    r'id="[^"]+_Circuit">.+?</ul>',
    html, re.DOTALL
)

result = []
for block in blocks:
    urls = re.findall(
         r'<i><a href="([^"]+)',
         block
    )
    circuit = re.findall(
             r'_Circuit">([^<]+)',
             block
        )
    for u in urls:
        time.sleep(2)
        a = 'https://ballotpedia.org'+u
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers)
        
        assert response.status_code == 200, 'Статус респонса не 200'
        
        html = response.content.decode()
        a = 'https://ballotpedia.org'+u
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(a, headers=headers)
                
        assert response.status_code == 200, 'Статус респонса не 200'
                
        html = response.content.decode()
        name = re.findall(
                 r'<table class="infobox".+<big><i><b>([^<]+)',
                 html, re.DOTALL
            )
        num = re.findall(
                 r'<b>Docket number:</b> ([^<]+)',
                 html, re.DOTALL
            )
        term = re.findall(
                 r'<b>Term:</b>([^<]+)',
                 html, re.DOTALL
            )
        date = re.findall(
                 r'Important dates</b></td></tr><tr><td colspan="2" style="text-align:center;">([^<]+)',
                 html, re.DOTALL
            )
        petitioner = re.findall(
                 r'<li><b>Petitioner</b>:([^<]+)',
                 html, re.DOTALL
            )
        respondent = re.findall(
                 r'<li><b>Respondent</b>:([^<]+)',
                 html, re.DOTALL
            )
        row = {
            'circuit': circuit[0], 
            'name':  name[0] if name else None, 
            'num': num[0] if num else None, 
            'term': term[0] if term else None,
            'date': date[0] if date else None,
            'petitioner': petitioner[0] if petitioner else None,
            'respondent': respondent[0] if respondent else None,
        }
        print(row)
        result.append(row)

{'circuit': '1st Circuit', 'name': 'Smith &amp; Wesson Brands v. Estados Unidos Mexicanos', 'num': '23-1141', 'term': ' 2024', 'date': 'Argument: March 4, 2025', 'petitioner': ' Smith &amp; Wesson Brands, Inc., ', 'respondent': ' Estados Unidos Mexicanos\n'}
{'circuit': '2nd Circuit', 'name': 'Medical Marijuana, Inc. v. Horn', 'num': '23-365', 'term': ' 2024', 'date': 'Argued: October 15, 2024', 'petitioner': ' Medical Marijuana, Inc., et al.\n', 'respondent': ' Douglas J. Horn\n'}
{'circuit': '2nd Circuit', 'name': 'Delligatti v. United States', 'num': '23-825', 'term': ' 2024', 'date': 'Argued: November 12, 2024', 'petitioner': ' Salvatore Delligatti\n', 'respondent': ' United States\n'}
{'circuit': '2nd Circuit', 'name': 'Cunningham v. Cornell University', 'num': '23-1007', 'term': ' 2024', 'date': 'Argued: January 22, 2025', 'petitioner': ' Casey Cunningham\n', 'respondent': ' Cornell University\n'}
{'circuit': '2nd Circuit', 'name': 'BLOM Bank SAL v. Honickman', 'num': '23-1259', 

In [163]:
df = pd.DataFrame(result)

In [3]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0'
headers = {'User-Agent': user_agent}

logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)
        
handler = logging.StreamHandler()
FORMAT = '%(asctime)s %(levelname)-8s %(message)s'
formatter = logging.Formatter(fmt=FORMAT)
handler.setFormatter(formatter)
logger.addHandler(handler)

class Parser:

    base_url = 'https://ballotpedia.org'
    doc_url = 'https://ballotpedia.org/Supreme_Court_cases,_October_term_2024-2025'
    pattern_circuit = r'_Circuit">([^<]+)'
    pattern_term = r'<b>Term:</b>([^<]+)'
    pattern_date = r'Important dates</b></td></tr><tr><td colspan="2" style="text-align:center;">([^<]+)'
    pattern_num = r'<b>Docket number:</b> ([^<]+)'
    pattern_urls = r'<i><a href="([^"]+)'
    pattern_name = r'<table class="infobox".+<big><i><b>([^<]+)'
    pattern_blocks = r'id="[^"]+_Circuit">.+?</ul>'
    pattern_petitioner = r'<li><b>Petitioner</b>:([^<]+)'
    pattern_respondent = r'<li><b>Respondent</b>:([^<]+)'
    
    def __init__(self):
        self.result = []

    def get_html(self, url, encoding='utf-8'):
        time.sleep(2)
        response = requests.get(url, headers=headers)
        assert response.status_code == 200, 'Статус респонса не 200'
        html = response.content.decode(encoding)
        return html

    def parse(self):
        html = self.get_html(self.doc_url)
        blocks = re.findall(self.pattern_blocks, html, re.DOTALL)
        for block in blocks:
            urls = re.findall(self.pattern_urls, block)
            circuit = re.findall(self.pattern_circuit, block)
            for u in urls:
                time.sleep(2)
                url = self.base_url + u
                logger.info(f'circuit: {circuit}, url: {url}')
                self.parse_doc(url, circuit)
        df = pd.DataFrame(self.result)
        return df

    def parse_doc(self, url, circuit):
        html = self.get_html(url)
        name = re.findall(self.pattern_name, html, re.DOTALL)
        num = re.findall(self.pattern_num, html, re.DOTALL)
        term = re.findall(self.pattern_term, html, re.DOTALL)
        date = re.findall(self.pattern_date, html, re.DOTALL)
        petitioner = re.findall(self.pattern_petitioner, html, re.DOTALL)
        respondent = re.findall(self.pattern_respondent, html, re.DOTALL)
        row = {
            'circuit': circuit[0], 
            'name':  name[0] if name else None, 
            'num': num[0] if num else None, 
            'term': term[0] if term else None,
            'date': date[0] if date else None,
            'petitioner': petitioner[0] if petitioner else None,
            'respondent': respondent[0] if respondent else None,
        }
        self.result.append(row)

In [5]:
parser = Parser()
res = parser.parse()
res

2025-02-16 20:52:00,129 INFO     circuit: ['1st Circuit'], url: https://ballotpedia.org/Smith_%26_Wesson_Brands_v._Estados_Unidos_Mexicanos
2025-02-16 20:52:05,172 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Medical_Marijuana,_Inc._v._Horn
2025-02-16 20:52:10,126 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Delligatti_v._United_States
2025-02-16 20:52:15,123 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Cunningham_v._Cornell_University
2025-02-16 20:52:20,824 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/BLOM_Bank_SAL_v._Honickman
2025-02-16 20:52:26,216 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Fuld_v._Palestine_Liberation_Organization
2025-02-16 20:52:31,869 INFO     circuit: ['3rd Circuit'], url: https://ballotpedia.org/Kousisis_v._United_States
2025-02-16 20:52:37,524 INFO     circuit: ['3rd Circuit'], url: https://ballotpedia.org/Commissioner_of_Internal_Revenue_v._Zuch
2025-02-16 20:5

,circuit,name,num,term,date,petitioner,respondent
0,1st Circuit,Smith &amp; Wesson Brands v. Estados Unidos Me...,23-1141,2024,"Argument: March 4, 2025","Smith &amp; Wesson Brands, Inc.,",Estados Unidos Mexicanos\n
1,2nd Circuit,"Medical Marijuana, Inc. v. Horn",23-365,2024,"Argued: October 15, 2024","Medical Marijuana, Inc., et al.\n",Douglas J. Horn\n
2,2nd Circuit,Delligatti v. United States,23-825,2024,"Argued: November 12, 2024",Salvatore Delligatti\n,United States\n
3,2nd Circuit,Cunningham v. Cornell University,23-1007,2024,"Argued: January 22, 2025",Casey Cunningham\n,Cornell University\n
4,2nd Circuit,BLOM Bank SAL v. Honickman,23-1259,2024,"Argument: March 3, 2025",BLOM Bank SAL\n,"Michal Honickman,"
5,2nd Circuit,Fuld v. Palestine Liberation Organization,24-20,2024,Pending,"Miriam Fuld, et al.\n","Palestine Liberation Organization, et al.\n"
6,3rd Circuit,Kousisis v. United States,23-909,2024,"Argued: December 9, 2024",None,None
7,3rd Circuit,Commissioner of Internal Revenue v. Zuch,24-416,2024,Pending,Commissioner of Internal Revenue\n,Jennifer Zuch\n
8,4th Circuit,Lackey v. Stinnie,23-621,2024,"Argued: October 8, 2024","Gerald F. Lackey, in His Official Capacity as...","Damian Stinnie, et al.\n"
9,4th Circuit,"E.M.D. Sales, Inc. v. Carrera",None,2024,None,None,None


In [6]:
api_headers = {
    # 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    # 'Accept': 'application/json',
    'X-Requested-With': 'XMLHttpRequest',
}


r = requests.get('https://fantasyscotus.net/case/list/?filterscount=0&groupscount=0&pagenum=0&pagesize=20&recordstartindex=0&recordendindex=12', 
                headers=api_headers)

r.json()

{'object_list': [{'docket_url': '/case/bouarfa-v-mayorkas/',
   'outcome': 'Affirm 9-0',
   'accuracy': 'correct',
   'short_name': 'Bouarfa v. Mayorkas',
   'date_argued': '2024-10-15',
   'prediction': 'Affirm 8-1',
   'id': 780,
   'date_decided': '2024-12-10',
   'stats_url': '/case-prediction/bouarfa-v-mayorkas/',
   'predicted': '',
   'docket_number': '23-583',
   'm_accuracy': ''},
  {'docket_url': '/case/royal-canin-usa-v-wullschleger/',
   'outcome': 'Affirm 9-0',
   'accuracy': 'correct',
   'short_name': 'Royal Canin U.S.A. v. Wullschleger',
   'date_argued': '2024-10-07',
   'prediction': 'Affirm 9-0',
   'id': 775,
   'date_decided': '2025-01-15',
   'stats_url': '/case-prediction/royal-canin-usa-v-wullschleger/',
   'predicted': '',
   'docket_number': '23-677',
   'm_accuracy': ''},
  {'docket_url': '/case/emd-sales-v-carrera/',
   'outcome': 'Reverse 9-0',
   'accuracy': 'correct',
   'short_name': 'E.M.D. Sales v. Carrera',
   'date_argued': '2024-11-05',
   'predicti